In [1]:
import json
from typing import Dict
import re
import pprint

In [15]:
import sqlite3
con = sqlite3.connect('fda_spl_data.db')
cur = con.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS fda_data (
drug_id text PRIMARY KEY,
spl text,
adverse_reactions text,
alarms text,
boxed_warning text,
carcinogenesis_and_mutagenesis_and_impairment_of_fertility text,
clinical_pharmacology text,
contraindications text,
drug_and_or_laboratory_test_interactions text,
drug_interactions text,
general_precautions text,
geriatric_use text,
indications_and_usage text,
nonclinical_toxicology text,
nonteratogenic_effects text,
pediatric_use text,
pharmacodynamics text,
pharmacogenomics text,
pharmacokinetics text,
purpose text,
teratogenic_effects text,
use_in_specific_populations text,
warnings text,
warnings_and_cautions text,
brand_name text,
generic_name text,
substance_name text,
application_number text
)
"""

cur.execute(create_table_query)

In [3]:
data_files = [
    'drug-label-0001-of-0011.json',
    'drug-label-0002-of-0011.json',
    'drug-label-0003-of-0011.json',
    'drug-label-0004-of-0011.json',
    'drug-label-0005-of-0011.json',
    'drug-label-0006-of-0011.json',
    'drug-label-0007-of-0011.json',
    'drug-label-0008-of-0011.json',
    'drug-label-0009-of-0011.json',
    'drug-label-0010-of-0011.json',
    'drug-label-0011-of-0011.json',
]

def iterate_over_files(callback_fn):
    for data_file in data_files:
        with open(data_file, 'r') as f:
            file = json.load(f)
        for result in file['results']:
            callback_fn(result)

### Total record count

In [5]:
count = {'count': 0}
def get_total_count(result: Dict):
    count['count'] += 1
iterate_over_files(get_total_count)
print(count)

{'count': 200315}


### Get all key names in records

In [8]:
unique_columns = set()
def get_all_columns(result: Dict):
    unique_columns.update(set(result.keys()))
iterate_over_files(get_all_columns)
for column in sorted(list(unique_columns)):
    print(column)

abuse
abuse_table
accessories
active_ingredient
active_ingredient_table
adverse_reactions
adverse_reactions_table
alarms
alarms_table
animal_pharmacology_and_or_toxicology
animal_pharmacology_and_or_toxicology_table
ask_doctor
ask_doctor_or_pharmacist
ask_doctor_or_pharmacist_table
ask_doctor_table
assembly_or_installation_instructions
boxed_warning
boxed_warning_table
calibration_instructions
carcinogenesis_and_mutagenesis_and_impairment_of_fertility
carcinogenesis_and_mutagenesis_and_impairment_of_fertility_table
cleaning
clinical_pharmacology
clinical_pharmacology_table
clinical_studies
clinical_studies_table
compatible_accessories
components
components_table
contraindications
contraindications_table
controlled_substance
dependence
dependence_table
description
description_table
diagram_of_device
disposal_and_waste_handling
do_not_use
do_not_use_table
dosage_and_administration
dosage_and_administration_table
dosage_forms_and_strengths
dosage_forms_and_strengths_table
drug_abuse_and_d

In [19]:
DRUGSATFDA_CONCEPT_ID_RE = re.compile("(ANDA|NDA)(\d*)")


In [32]:
def load_record_in_db(result: Dict):
    drug = (
        result.get("id"),
        result.get("spl", ""),
        "|".join(result.get("adverse_reactions", [])),
        result.get("alarms", ""),
        result.get("boxed_warning", ""),
        result.get("carcinogenesis_and_mutagenesis_and_impairment_of_fertility", ""),
        result.get("clinical_pharmacology", ""),
        result.get("contraindications", ""),
        result.get("drug_and_or_laboratory_test_interactions", ""),
        result.get("drug_interactions", ""),
        result.get("general_precautions", ""),
        result.get("geriatric_use", ""),
        "|".join(result.get("indications_and_usage", [])),
        result.get("nonclinical_toxicology", ""),
        result.get("nonteratogenic_effects", ""),
        result.get("pediatric_use", ""),
        result.get("pharmacodynamics", ""),
        result.get("pharmacogenomics", ""),
        result.get("pharmacokinetics", ""),
        "|".join(result.get("purpose", [])),
        result.get("teratogenic_effects", ""),
        result.get("use_in_specific_populations", ""),
        "|".join(result.get("warnings", [])),
        result.get("warnings_and_cautions", ""),
        "|".join(result.get("openfda", {}).get("brand_name", [])),
        "|".join(result.get("openfda", {}).get("generic_name", [])),
        "|".join(result.get("openfda", {}).get("substance_name", [])),
        "|".join(result.get("openfda", {}).get("application_number", []))
    )
    
    query = """
    INSERT INTO fda_data(
        drug_id, 
        spl, 
        adverse_reactions, 
        alarms, 
        boxed_warning, 
        carcinogenesis_and_mutagenesis_and_impairment_of_fertility,
        clinical_pharmacology, 
        contraindications, 
        drug_and_or_laboratory_test_interactions, 
        drug_interactions, 
        general_precautions, 
        geriatric_use, 
        indications_and_usage, 
        nonclinical_toxicology, 
        nonteratogenic_effects, 
        pediatric_use, 
        pharmacodynamics, 
        pharmacogenomics, 
        pharmacokinetics, 
        purpose, 
        teratogenic_effects, 
        use_in_specific_populations, 
        warnings, 
        warnings_and_cautions, 
        brand_name, 
        generic_name, 
        substance_name, 
        application_number
    )
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """
    try:
        cur.execute(query, drug)
    except Exception:
        print(drug)

cur.execute("DELETE FROM fda_data")
iterate_over_files(load_record_in_db)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [42]:
cur.execute("SELECT brand_name, indications_and_usage FROM fda_data WHERE indications_and_usage != '' LIMIT 50").fetchall()

[('SENSODYNE PRONAMEL Intensive Enamel Repair Whitening',
  'Uses • builds increasing protection against painful sensitivity of the teeth to cold, heat, acids, sweets, or contact. • aids in the prevention of dental cavities.'),
 ('Hal-Hen Earwax Removal System',
  'Uses for occasional use as an aid to soften, loosen, and remove excessive earwax'),
 ('Lil Drug Store Headache Relief Extra Strength',
  'Uses temporarily relieves minor aches and pains due to: headache a cold arthritis muscular aches toothache premenstrual and menstrual cramps'),
 ('MAXIMUM STRENGTH HEMORRHOIDAL RELIEF',
  'Use helps relieve the pain, itching and burning associated with hemorrhoids and other anorectal disorders.'),
 ('',
  'Uses Temporarily relieves the following symptoms associated with hay fever or other upper respiratory allergies runny nose sneezing itching of the nose or throat itchy, watery eyes'),
 ('', 'Uses Helps prevent sunburn.'),
 ('SHISEIDO SYNCHRO SKIN LASTING Liquid FOUNDATION Neutral 5',
  '